<a href="https://colab.research.google.com/github/tinkercademy/ml-notebooks/blob/main/Generative AI/05_GAN_Example_Program_Digit_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torch import nn

import math
import matplotlib.pyplot as plt
import torchvision
import torchvision.transforms as transforms

from tqdm import tqdm

Sets the seed of any random number generating function, so that the generated numbers are reproducible.

e.g. torch.rand(2) will consistently give you tensor([0.7156, 0.9140]) with torch.manual_seed(111)

In [2]:
torch.manual_seed(121)

Uses GPU if it's available, and CPU otherwise. (GPU would be faster)

In [3]:
device = ""
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

Transformation required to convert the dataset to a pytorch Tensor, and to set the range of coefficients to be from -1 to 1.

The output of Normalize(mean, std) = (Tensor input - mean) / std

In [4]:
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))]
)

Downloads the MNIST dataset from pytorch, after transforming it into the required format.

In [ ]:
train_set = torchvision.datasets.MNIST(
    root=".", train=True, download=True, transform=transform
)

Loads the dataset into a pytorch dataloader to obtain shuffled batches of 32 samples. The batch size is the number of samples we pass into the training loop at each iteration, while shuffle is useful to have real and generated data in each batch.

In [6]:
batch_size = 32
train_loader = torch.utils.data.DataLoader(
    train_set, batch_size=batch_size, shuffle=True
)

In [ ]:
real_samples, mnist_labels = next(iter(train_loader))
for i in range(16):
    ax = plt.subplot(4, 4, i + 1)
    plt.imshow(real_samples[i].reshape(28, 28), cmap="gray_r")
    plt.xticks([])
    plt.yticks([])

Our Discriminator model which contains the neural network in self.model.

The forward method of the Discriminator works the same way \_\_call\_\_ does for a regular python class (i.e. runs when Discriminator(arg) is called).

In [12]:
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(784, 1024),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 1),
            nn.Sigmoid(),
        )

    def forward(self, x):
        x = x.view(x.size(0), 784)
        output = self.model(x)
        return output

In [13]:
discriminator = Discriminator().to(device=device)

Our Generator model which contains the neural network in self.model.

The forward method of the Generator works the same way \_\_call\_\_ does for a regular python class (i.e. runs when Generator(arg) is called).

In [14]:
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(100, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 1024),
            nn.ReLU(),
            nn.Linear(1024, 784),
            nn.Tanh(),
        )

    def forward(self, x):
        output = self.model(x)
        output = output.view(x.size(0), 1, 28, 28)
        return output

generator = Generator().to(device=device)

_lr_ refers to the learning rate of the model.

_num\_epochs_ refers to how many repetitions of training will be done using the whole training set.

_loss\_function_ refers to the loss function (i.e. binary cross-entropy) used by the model.

In [15]:
lr = 0.0001
num_epochs = 50
loss_function = nn.BCELoss()


optimizer_discriminator = torch.optim.Adam(discriminator.parameters(), lr=lr)
optimizer_generator = torch.optim.Adam(generator.parameters(), lr=lr)


The standard way of organising data in pytorch is to have each line of the tensor to represent one sample from the batch.

torch.ones(_size_) returns a tensor filled with scalar value 1, with shape of argument _size_. This is to represent they are real images.

latent\_space\_samples represents a tensor filled with random numbers, of size (batch_size, 100). 100 matches the input size of the Generator input, it will be used as a random input to generate.

torch.zeros(_size_) returns a tensor filled with scalar value 0, with shape of argument _size_. This is to represent they are generated images.

torch.cat((a, b)) concatenates tensors a and b.

discriminator.zero_grad() is required to clear the gradients at each training step.

loss_discriminator.backward() calculates gradients to update the weights. optimizer_discriminator.step() actually updates the discriminator weights. Same for generator. This is backpropagation.

In [ ]:
for epoch in range(num_epochs):
    for n, (real_samples, mnist_labels) in enumerate(tqdm(train_loader)):
        # Data for training the discriminator
        real_samples = real_samples.to(device=device)
        real_samples_labels = torch.ones((batch_size, 1)).to(
            device=device
        )

        # Generate random noise (tensors of random values) as input for the Generator
        latent_space_samples = torch.randn((batch_size, 100)).to(
            device=device
        )

        # We get generated samples to train the discriminator with
        generated_samples = generator(latent_space_samples)
        generated_samples_labels = torch.zeros((batch_size, 1)).to(
            device=device
        )

        # We concatenate the real and generated samples into one group
        all_samples = torch.cat((real_samples, generated_samples))
        all_samples_labels = torch.cat(
            (real_samples_labels, generated_samples_labels)
        )

        # Training the discriminator
        discriminator.zero_grad()
        output_discriminator = discriminator(all_samples)
        loss_discriminator = loss_function(
            output_discriminator, all_samples_labels
        )
        loss_discriminator.backward()
        optimizer_discriminator.step()

        # Data for training the generator, we generate random noise as input again
        latent_space_samples = torch.randn((batch_size, 100)).to(
            device=device
        )

        # Training the generator
        generator.zero_grad()
        generated_samples = generator(latent_space_samples)
        output_discriminator_generated = discriminator(generated_samples)
        loss_generator = loss_function(
            output_discriminator_generated, real_samples_labels
        )
        loss_generator.backward()
        optimizer_generator.step()

        # Show loss
        if n == batch_size - 1:
            print(f"Epoch: {epoch} Loss D.: {loss_discriminator}")
            print(f"Epoch: {epoch} Loss G.: {loss_generator}")


In [17]:
# Training is over, time to generating random noise as input for Generator to produce the digits!
latent_space_samples = torch.randn(batch_size, 100).to(device=device)
generated_samples = generator(latent_space_samples)

In [ ]:
generated_samples = generated_samples.cpu().detach()
for i in range(16):
    ax = plt.subplot(4, 4, i + 1)
    plt.imshow(generated_samples[i].reshape(28, 28), cmap="gray_r")
    plt.xticks([])
    plt.yticks([])